In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import pickle
with open('../data/notion_offline.pkl', 'rb') as f:
    docs_from_notion = pickle.load(f)

In [4]:
from typing import Dict, List
from langchain_core.documents.base import Document

def process_property(docs_from_notion: Dict[str, List[Document]]) -> List[Document]:
    docs_list = list()
    
    for db_name, docs in docs_from_notion.items():
        for doc in docs:
            # because our data are gathered from multiple databases
            # we are going to throw the database names as one property
            # into the docs' metadata field
            # and return as a list
            doc.metadata['source'] = db_name

            # change dates into YYYYMMDD int format to allow GT/LT/EQ comparison
            if 'date' in doc.metadata:
                if 'start' in doc.metadata['date']:
                    doc.metadata['date_start'] = int(doc.metadata['date']['start'].replace("-", ""))
                if 'end' in doc.metadata['date'] and doc.metadata['date']['end']:
                    doc.metadata['date_end'] = int(doc.metadata['date']['end'].replace("-", ""))
                    
                del doc.metadata['date']

            if 'tags' in doc.metadata:
                doc.metadata['tags'] = ", ".join(doc.metadata['tags'])
                

        docs_list.extend(docs)
        
    return docs_list

In [5]:
docs = process_property(docs_from_notion)

In [7]:
from my_notion_companion.document_filter import DocumentFilter

In [30]:
x = DocumentFilter(0.6)

In [31]:
_ = x.filter_by_metadata("读书笔记", docs)

2024-03-01 12:41:53.674 | INFO     | my_notion_companion.document_filter:filter_by_metadata:27 - Remaining doc:  0.609
